In [2]:
import numpy as np; import pandas as pd
import re; import datetime; from collections import Counter, OrderedDict
import category_encoders as ce
from sklearn.model_selection import train_test_split
import pylogit as pl

In [17]:
df = pd.read_csv(r'가마우지\race.csv', engine='python', encoding='utf-8')

In [75]:
df.columns

Index(['rank', 'lane', 'sex', 'age', 'jockey_w', 'dandivi', 'yeondivi',
       'cure_in_1m', 'weight_diff', 'raw_weight', 'weight_added', 'distance',
       'velocity', 'horse_level_num', 'groupid', 'adj_vel', 'prev_rank',
       'norating'],
      dtype='object')

In [19]:
df['groupid'] = df['date'] + df['round'].apply(str)

In [20]:
idset = list(set(df['groupid']))
give_ids = dict(list(zip(idset, list(range(len(idset))))))

In [21]:
df['groupid'] = df['groupid'].apply(lambda x: give_ids[x])
df = df[df['distance'] > 0] # 거리 정보 있는 것만 살림

In [34]:
distances = list(set(df['distance']))
avg_vel = {}=

In [42]:
for i in distances:
    avg_vel[i] = df[df['distance'] == i]['velocity'].mean()

In [48]:
df['avg_vel'] = df['distance'].apply(lambda x: avg_vel[x])
df['adj_vel'] = df['velocity'] / df['avg_vel']

In [67]:
nums = list(map(str, range(1,17)))
df = df[(df['prev1_rank'].apply(lambda x: x in nums)) & (df['prev2_rank'].apply(lambda x: x in nums))]

In [70]:
df['prev_rank'] = df['prev1_rank'] + df['prev2_rank']
del df['prev1_rank'], df['prev2_rank']

In [74]:
df['norating'] = (df['horse_level_num'] == 10).apply(int)

In [86]:
df['choice'] = (df['rank'] == 1).apply(int)

In [90]:
groupids = list(set(df['groupid']))
train, test = train_test_split(groupids, test_size = 0.3, random_state=datetime.datetime.now().second)

train = df[df['groupid'].apply(lambda x: x in train)]
test = df[df['groupid'].apply(lambda x: x in test)]

In [94]:
train2 = train[train['rank']>1]
train2['choice'] = (train2['rank'] == 2).apply(int)
train2['groupid'] = train2['groupid'] + 10000

train3 = train[train['rank']>2]
train3['choice'] = (train3['rank'] == 3).apply(int)
train3['groupid'] = train3['groupid'] + 20000

C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try u

In [96]:
train = train.append(train2, ignore_index=True).append(train3, ignore_index=True)

In [121]:
train.columns

Index(['lane', 'age', 'jockey_w', 'dandivi', 'yeondivi', 'cure_in_1m',
       'weight_diff', 'raw_weight', 'weight_added', 'horse_level_num',
       'groupid', 'prev_rank', 'norating', 'choice', 'intercept', 'sex1',
       'sex2'],
      dtype='object')

In [99]:
del train['rank'], train['distance'], train['velocity']

In [200]:
basic_specification = OrderedDict()
basic_names = OrderedDict()
li = list(range(1,17))
xs = ['lane', 'sex1', 'sex2', 'age', 'jockey_w', 'dandivi', 'yeondivi', 'cure_in_1m', 'raw_weight', 'weight_added', 'horse_level_num', 'prev_rank', 'norating']
basic_specification["intercept"] = [li]
basic_names['intercept'] = ['annoying']

for i in xs:
    basic_specification[i] = [li]
    basic_names[i] = [i]

In [135]:
custom_alt_id = "lane"
obs_id_column = "groupid"
choice_column = "choice"

In [202]:
clogit_mnl = pl.create_choice_model(data=train_new.sort_values(by=['groupid']),
                                        alt_id_col=custom_alt_id,
                                        obs_id_col=obs_id_column,
                                        choice_col=choice_column,
                                        specification=basic_specification,
                                        model_type="MNL",
                                        names=basic_names)

clogit_mnl.fit_mle(np.ones(14))
clogit_mnl.get_statsmodels_summary()

Log-likelihood at zero: -30,146.3572
Initial Log-likelihood: -1,138,601.4770


C:\Users\myeon\AppData\Local\Continuum\anaconda3\lib\site-packages\scipy\optimize\_minimize.py:420: RuntimeWarning: Method BFGS does not use Hessian information (hess).
  RuntimeWarning)


Estimation Time for Point Estimation: 3.30 seconds.
Final log-likelihood: -32,563.6512


MemoryError: 

In [6]:
test['sex1'] = (test['sex'] == '수').apply(int)
test['sex2'] = (test['sex'] == '암').apply(int)
del test['sex']

In [119]:
train = train[train['horse_level_num'] > 0]

In [5]:
def rank(x):
    y=str(x)
    if len(y) == 2:
        return int(y[0])+int(y[1])
    if len(y) == 3:
        return int(y[:2])+int(y[2])
    else:
        return int(y[:2])+int(y[2:])
test['prev_rank'] = test['prev_rank'].apply(rank)

In [175]:
groupids = list(set(train['groupid']))

In [189]:
import random
tmps = []; memory = []

for i in groupids:
    dta = train[train['groupid']==i]
    diff = list(dta['lane'])
    dta = dta[dta['choice'] == 0]
    if len(dta)==0: 
        memory.append(i)
        continue
    for j in li:
        if j not in diff:
            r = random.randint(0, len(dta)-1)
            tmp=list(dta.iloc[r,:])
            tmp[0]=j
            tmps.append(tmp)

In [194]:
train_new = train.append(pd.DataFrame(tmps, columns = train.columns), ignore_index = True)

In [198]:
train_new = train_new[train_new['식별'] == False]

In [196]:
train_new['식별'] = train_new['groupid'].apply(lambda x: x in memory)

In [199]:
train_new

,lane,age,jockey_w,dandivi,yeondivi,cure_in_1m,raw_weight,weight_added,horse_level_num,groupid,prev_rank,norating,choice,intercept,sex1,sex2,식별
0,10,4.0,54.0,1.9,1.2,0.0,447.0,-7.0,3.0,1136.0,8.0,0.0,1.0,1.0,1.0,0.0,False
1,3,6.0,54.5,40.2,8.1,0.0,480.0,-1.0,3.0,1136.0,7.0,0.0,0.0,1.0,0.0,0.0,False
2,11,3.0,55.5,4.7,2.0,1.0,463.0,-5.0,3.0,1136.0,6.0,0.0,0.0,1.0,0.0,1.0,False
3,6,3.0,52.0,14.5,3.6,1.0,518.0,7.0,3.0,1136.0,5.0,0.0,0.0,1.0,0.0,0.0,False
4,2,3.0,52.0,13.8,3.6,1.0,512.0,-1.0,3.0,1136.0,9.0,0.0,0.0,1.0,0.0,0.0,False
5,4,3.0,52.0,7.5,2.3,0.0,479.0,-13.0,3.0,1136.0,9.0,0.0,0.0,1.0,1.0,0.0,False
6,12,5.0,52.5,27.8,4.2,0.0,468.0,-21.0,3.0,1136.0,8.0,0.0,0.0,1.0,1.0,0.0,False
7,1,3.0,56.0,10.9,2.5,1.0,462.0,0.0,3.0,1136.0,13.0,0.0,0.0,1.0,0.0,0.0,False
8,9,5.0,53.5,22.3,3.4,0.0,470.0,-1.0,3.0,1136.0,9.0,0.0,0.0,1.0,0.0,0.0,False
9,5,3.0,52.0,69.5,11.8,0.0,497.0,8.0,3.0,1136.0,16.0,0.0,0.0,1.0,0.0,0.0,False


In [203]:
train.to_csv('clogit_train.csv', encoding='utf-8', index=False)
train_new.to_csv('clogit_train_new.csv', encoding='utf-8', index=False)
test.to_csv('clogit_test.csv', encoding='utf-8', index=False)

In [1]:
import pandas as pd
test = pd.read_csv('clogit_test.csv', encoding='utf-8')

In [10]:
del test['weight_diff'], test['distance'], test['velocity'], test['adj_vel']

In [24]:
test.to_csv('clogit_test.csv', encoding='utf-8', index=False)

PermissionError: [Errno 13] Permission denied: 'clogit_test.csv'

In [3]:
test['phat']=pd.read_csv('dfdsaf.csv', header=None)

In [6]:
test = test.sort_values(by=['groupid'])

In [7]:
test['delete']=test['phat'].apply(lambda x: x == '-')
test = test[test['delete'] == 0]

In [63]:
test['phat']=test['phat'].apply(float)

In [80]:
idx = list(set(test['groupid']))
dictt=dict(test.groupby(['groupid'])['phat'].max())
dandivi = []

t=0
for i in idx:
    if list(test[(test['groupid']==i) & (test['phat']==dictt[i])]['choice'])[0] == 1:
        t = t + 1
        dandivi.append(list(test[(test['groupid']==i) & (test['phat']==dictt[i])]['dandivi'])[0])

In [85]:
import numpy as np
(np.array(dandivi) >= 3).mean()

0.292894280762565

In [70]:
t / test.groupby(['groupid']).max()['choice'].sum()

0.3705844572896596

In [26]:
for_accu=dict(test.groupby(['groupid']).count()['lane']<5) # 105 그룹 있음.

In [37]:
test['foraccu'] = test['groupid'].apply(lambda x: for_accu[x])

In [41]:
test_n = test[test['foraccu']==False]

In [71]:
idx = list(set(test_n['groupid']))

t=0
for i in idx:
    if list(test_n[(test_n['groupid']==i) & (test_n['phat']==dictt[i])]['choice'])[0] == 1:
        t = t + 1
        
t / test_n.groupby(['groupid']).max()['choice'].sum()

0.3623283191628515

In [66]:
dictt

{8: 0.212596953,
 12: 0.28463012,
 14: 0.266750008,
 15: 0.375189483,
 17: 0.208918601,
 23: 0.313079655,
 26: 0.225631297,
 29: 0.299129218,
 30: 0.209057167,
 31: 0.212362871,
 33: 0.203817129,
 37: 0.188448161,
 45: 0.172373652,
 46: 0.213428959,
 47: 0.226557866,
 49: 0.169990361,
 52: 0.267900854,
 53: 0.159644932,
 63: 0.252589136,
 64: 0.145869598,
 65: 0.283684194,
 67: 0.21078968,
 72: 0.189326569,
 73: 0.211305484,
 74: 0.192271486,
 75: 0.257783592,
 78: 0.169595912,
 80: 0.251829624,
 84: 0.312298059,
 85: 0.222116008,
 89: 0.274394572,
 93: 0.159478486,
 95: 0.178397298,
 97: 0.235375255,
 98: 0.202106029,
 100: 0.204954892,
 101: 0.388324529,
 105: 0.916212976,
 106: 0.227166355,
 107: 0.20772104,
 111: 0.290384829,
 112: 0.265069336,
 114: 0.290498078,
 115: 0.251001328,
 118: 0.200163797,
 125: 1.0,
 126: 0.295888692,
 127: 0.215474382,
 130: 0.236369058,
 138: 0.194761902,
 140: 0.292485386,
 141: 0.206802264,
 149: 0.288844496,
 151: 0.276112378,
 156: 0.308695406,
 1

In [68]:
test.head()

,lane,age,jockey_w,dandivi,yeondivi,cure_in_1m,raw_weight,weight_added,horse_level_num,groupid,prev_rank,norating,choice,sex1,sex2,phat,delete,foraccu
5728,6,3,51.0,5.8,1.7,1,459.0,-7.0,3.0,8,12,0,0,0,1,0.124957,False,False
5718,12,3,53.5,2.0,1.3,0,482.0,2.0,3.0,8,9,0,1,0,1,0.212597,False,False
5719,10,3,51.5,9.0,1.8,0,465.0,-2.0,3.0,8,11,0,0,0,0,0.205747,False,False
5720,5,4,52.5,9.6,2.5,0,443.0,-4.0,3.0,8,9,0,0,0,1,0.131718,False,False
5721,8,5,56.5,36.9,5.6,0,486.0,18.0,3.0,8,13,0,0,1,0,0.045573,False,False


In [47]:
test_n.groupby(['groupid']).max() # max method가 이상함.

,lane,age,jockey_w,dandivi,yeondivi,cure_in_1m,raw_weight,weight_added,horse_level_num,prev_rank,norating,choice,sex1,sex2,phat,delete,foraccu
groupid,,,,,,,,,,,,,,,,,
8,12,5,56.5,114.0,18.0,1,517.0,18.0,3.0,81,0,1,1,1,0.212596953,False,False
12,13,5,58.0,152.8,42.2,1,523.0,9.0,3.0,81,0,1,1,1,1.30E-05,False,False
14,10,7,56.0,161.3,11.6,1,495.0,14.0,2.0,85,0,1,1,1,0.266750008,False,False
15,11,6,56.5,64.0,11.2,1,479.0,10.0,2.0,91,0,1,1,1,0.375189483,False,False
17,12,4,57.0,85.0,15.5,1,524.0,6.0,4.0,94,0,1,1,0,0.208918601,False,False
23,14,7,57.0,136.4,32.1,1,544.0,6.0,2.0,93,0,1,1,1,0.313079655,False,False
26,12,5,56.5,102.4,21.4,1,500.0,0.0,5.0,81,0,1,1,1,0.225631297,False,False
29,12,5,56.0,82.3,23.1,1,499.0,10.0,4.0,93,0,1,1,1,0.299129218,False,False
30,11,5,55.0,66.7,12.3,1,564.0,9.0,5.0,23,0,1,1,1,0.209057167,False,False
